In [ ]:
import gensim
import numpy as np
import string
import tensorflow as tf
import tensorflow.contrib.slim as slim
import Placeholders
import datetime
import random as random

In [ ]:
x = Placeholders.sentence
y_ = Placeholders.class_labels
c = Placeholders.class_centroids

In [ ]:
def get_cluster_based_labels(fc_layer_representation):
    indexed_centroids = tf.matmul(y_,c)
    print(indexed_centroids)
    x_distances = tf.exp(-(tf.square(fc_layer_representation - indexed_centroids)))
    sum_x_distances = tf.log(tf.reduce_sum(x_distances))
    logits = tf.divide(x_distances, sum_x_distances)
    return logits

def get_cluster_based_labels1(fc_layer_representation):
    centroid0_distances = -tf.reduce_sum(tf.square(fc_layer_representation - c[0]),1)
    centroid1_distances = -tf.reduce_sum(tf.square(fc_layer_representation - c[1]),1)
    print("centroid:",tf.shape(centroid1_distances))
    x_distances = tf.reshape(tf.stack([centroid0_distances,centroid1_distances]),[-1,2])
    print("centroid:",tf.shape(x_distances))
    #return x_distances
    sum_x_distances = tf.reduce_sum(x_distances,1)
    sum_x_distances = tf.reshape(sum_x_distances, (-1, 1))
    logits = tf.divide(x_distances, sum_x_distances)
    return logits

def get_cluster_distance_loss(fc_layer_representation, centroid):
    x_distances = tf.square(fc_layer_representation - centroid)
    mean_x_distances = tf.reduce_mean(x_distances)
    return mean_x_distances
    

In [ ]:
def one_hot(vec, vals=Placeholders.n_classes):
    n = vec.shape[1]
    out = np.zeros((n, vals))
    out[np.arange(n), vec] = 1
    return out

In [ ]:
def get_random_examples(data,labels,number_of_examples,label=0):
    vector_encoded_labels = np.argmax(labels, axis=1)
    requested_labeled_examples = data[vector_encoded_labels==label,:]
    total_examples = requested_labeled_examples.shape[0]
    random_index = random.randint(0,total_examples - number_of_examples - 1)
    result = requested_labeled_examples[random_index:random_index+number_of_examples,:]
    return result
    

In [ ]:
def get_centroid(support_examples,hidden2,sess):
    hidden_vectors = sess.run(hidden2, feed_dict={x: support_examples})
    centroid = np.mean(hidden_vectors,axis = 0)
    return centroid
    

In [ ]:
def train(sess):
    he_init = slim.variance_scaling_initializer()
    hidden1 =tf.layers.dense(Placeholders.sentence, Placeholders.hidden1_neurons,activation=tf.nn.relu,kernel_initializer=he_init)
    hidden2 =tf.layers.dense(Placeholders.sentence, Placeholders.hidden2_neurons,activation=tf.nn.relu,kernel_initializer=he_init)
    
    #Cluster based objective
    cluster0_loss = get_cluster_distance_loss(hidden2,c[0])
    cluster1_loss = get_cluster_distance_loss(hidden2,c[1])
    logits = get_cluster_based_labels(hidden2)
    
    ##Basic Neural Network
    #cluster_based_labels = hidden2
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits= logits, labels=Placeholders.class_labels)
    loss = tf.reduce_mean(cross_entropy)
    #loss = cluster0_loss + cluster1_loss
    train_step = tf.train.AdamOptimizer(1e-5).minimize(loss)

    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Placeholders.class_labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    STEPS = 100000
    MINIBATCH_SIZE = 20
    
    print ("Starting at:" , datetime.datetime.now())
    sess.run(tf.global_variables_initializer())
    print ("Initialization done at:" , datetime.datetime.now())
    
    n_samples = 100
    mu, sigma = 10, 10
    X = np.array([np.random.normal(mu, sigma, Placeholders.word_vec_dim) for i in range(n_samples)])
    Y = np.zeros(n_samples, dtype=np.int16)
    X_validation= np.array([np.random.normal(mu, sigma, Placeholders.word_vec_dim) for i in range(n_samples)])
    Y_validation = np.zeros(n_samples, dtype=np.int16)
    X_test = np.array([np.random.normal(mu, sigma, Placeholders.word_vec_dim) for i in range(n_samples)])
    Y_test = np.zeros(n_samples, dtype=np.int16)
    mu, sigma = 50, 10
    X = np.vstack((X, np.array([np.random.normal(mu, sigma, Placeholders.word_vec_dim) for i in range(n_samples)])))
    Y = np.vstack((Y,np.ones(n_samples,dtype=np.int16))).reshape(-1,n_samples *2 )
    X_test = np.vstack((X_test, np.array([np.random.normal(mu, sigma, Placeholders.word_vec_dim) for i in range(n_samples)])))
    Y_test = np.vstack((Y_test,np.ones(n_samples,dtype=np.int16))).reshape(-1,n_samples *2 )
    X_validation = np.vstack((X_validation, np.array([np.random.normal(mu, sigma, Placeholders.word_vec_dim) for i in range(n_samples)])))
    Y_validation = np.vstack((Y_validation,np.ones(n_samples,dtype=np.int16))).reshape(-1,n_samples *2 )
    #print(Y)
    Y = one_hot(Y)
    Y_test= one_hot(Y_test)
    Y_validation= one_hot(Y_validation)
    #print (X.shape,Y.shape)
    C = np.ones(Placeholders.hidden2_neurons)*10
    C = np.vstack((C,np.ones(Placeholders.hidden2_neurons)*50)).reshape(-1,Placeholders.hidden2_neurons)
    
    processed_set = X
    
    SUPPORT_SIZE = 10
    QUERY_SIZE = 20
    
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for i in range(STEPS):
        #Get the QUERY SET 
        random_index = random.randint(0,n_samples * 2 - QUERY_SIZE - 1)
        random_X = X[random_index : random_index + QUERY_SIZE,:].reshape(-1, Placeholders.word_vec_dim)
        random_Y = Y[random_index : random_index + QUERY_SIZE,:].reshape(-1, Placeholders.n_classes)

        random_support_positive_examples = get_random_examples(X,Y,SUPPORT_SIZE,1)
        random_support_negative_examples = get_random_examples(X,Y,SUPPORT_SIZE,0)
        
        c_zero = get_centroid(random_support_negative_examples,hidden2,sess)
        c_one = get_centroid(random_support_positive_examples,hidden2,sess)
        C = np.vstack((c_zero,c_one))
        
        sess.run(train_step, feed_dict={x: random_X, y_: random_Y, c:C})
        
        if(i%2500 == 0):
            #logits = sess.run(logits, feed_dict={x: random_X, y_: random_Y, c:C})
            #print(logits)
            training_accuracy = sess.run(accuracy, feed_dict={x: random_X, y_: random_Y, c:C})*100
            training_loss = sess.run(loss, feed_dict={x: random_X, y_: random_Y, c:C})
            print("Training Accuracy: {0:.2f}".format(training_accuracy), " Training Loss: {0:.2f}".format(training_loss))
            validation_accuracy = sess.run(accuracy, feed_dict={x: X_validation, y_: Y_validation, c:C})*100
            test_accuracy = sess.run(accuracy, feed_dict={x: X_test, y_: Y_test, c:C})*100
            print("Validation Accuracy: {0:.2f}".format(validation_accuracy), " Test Accuracy: {0:.2f}".format(test_accuracy))

    

In [ ]:
with tf.Session() as sess:
    train(sess)